# Introdução ao Reconhecimento de Padrões, 2020.2, UFC/DETI
## Trabalho 1

Aluno : Thyago Freitas da Silva <br>
Matrícula : 392035

In [198]:
import numpy as np
import operator
import collections
import pandas as pd
from random import randrange
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### Funções úteis durante o decorrer do relatório

#### Transformar coluna contendo valores inválidos, substituindo-os pela mediana da coluna

In [199]:
def transform_column(column):
    transformed_column = np.array([0]*len(column))
    values = []
    for v in column:
        if v != '?':
            v = float(v)
            values.append(v)
    median = np.median(values)
    for index in range(len(column)):
        if column[index] == '?':
            transformed_column[index] = median
        else:
            transformed_column[index] = float(column[index])
    return transformed_column

#### Calcular acurácia

In [200]:
def accuracy_score(prediction,real_values):
    size = len(real_values)
    corrects = 0
    for index in range(size):
        if prediction[index] == real_values[index]:
            corrects += 1
    return corrects/size

#### Calcular acurácia por classe

In [201]:
def accuracy_score_per_class(prediction,real_values,classe):
    size = len(real_values)
    corrects = 0
    total = 0
    for index in range(size):
        if real_values[index] == classe:
            total += 1
        if prediction[index] == real_values[index] == classe:
            corrects += 1
    return corrects/total

#### Calcular matriz de confusão

In [202]:
def confusion_matrix(predict,real_values):
    classes = set(real_values)
    n_predicts = len(predict)
    n_classes = len(classes)
    confusion_m = np.zeros((n_classes,n_classes))
    for cl in classes:
        for index in range(n_predicts):
            if real_values[index] == cl:
                confusion_m[int(predict[index])-1,int(real_values[index])-1] += 1
    return confusion_m

#### Calcular o discriminante para o LDA

In [203]:
def discrimant(mean_per_class,within_classes,new_data,classes,class_value,n_features):
    count_instances = np.count_nonzero(classes == class_value)
    first_term = np.dot(mean_per_class,np.linalg.inv(within_classes))
    mul_transpose = np.dot(first_term,new_data)
    second_term = np.dot(0.5*first_term,np.transpose(mean_per_class))
    return mul_transpose - second_term + np.log(count_instances/n_features)

#### Calcular matriz de espalhamento

In [204]:
def within_classes(features,cov_matrixes,classes):
    dim = features.shape[1]
    rows = features.shape[0]
    w_c = np.zeros((dim,dim))
    values = []
    for cov_idx,cov in enumerate(cov_matrixes):
        n_instances_of_class = np.count_nonzero(classes == cov_idx)
        term = (n_instances_of_class/rows)*cov
        values.append(term)
    for v in values:
        w_c = np.add(w_c,v)
    return w_c

#### Matriz de covariância para uma classe específica

In [205]:
def cov_matrix(features,classes,class_value,mean):
    columns = features.shape[1]
    rows = classes == class_value
    matriz = features[rows]
    for row in matriz:
        for c in range(columns):
            row[c] -= mean[c]
    n_instances_of_class = np.count_nonzero(classes == class_value)
    cov = (1/n_instances_of_class)*np.dot(np.transpose(matriz),matriz)
    return cov

#### Calcular o vetor médiro de uma matriz

In [206]:
def feature_mean(features):
    return np.mean(features, axis = 0)

## Parte 0 : Banco de dados - Demartology

O banco de dados utilizado foi o "Demartology" que possui as seguintes características:

<ul>
<li>O arquivo CSV com os dados possui 35 colunas, sendo que a última coluna representa de forma numérica uma doenção demartológica.</li>
<li>A coluna 33 (band-like infiltrate) apresenta valores inválidos e que precisam ser removidos ou processados.</li>
<li>Temos 366 amostras no banco de dados.</li>
<li>A coluna 34 representa uma doença demartológica de forma numérica(1 à 6), sendo elas :</li>
    <ol>
        <li>psoriasis</li>
        <li>seboreic dermatitis</li>
        <li>lichen planus</li>
        <li>pityriasis rosea</li>
        <li>cronic dermatitis</li>
        <li>pityriasis rubra pilaris</li>
    </ol>
</ul>


In [207]:
data = pandas.read_csv("./data/dermatology.csv", header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3


## Parte 1 : KNearest Neighbors
### Funcionamento do algoritmo

O algoritmo conhecido como KNN (K-Nearest Neighbors) é um dos muitos algoritmos para classificação, mas que também possui uma versão que pode ser utilizada para regressão, e um dos primeiros a serem apresentados a iniciantes na área de aprendizado de máquina. Seu funcionamento pode ser descrito nas seguintes etapas.

<ol>
<li>Recebe uma amostra que ainda não foi classificada.</li>
<li>Calcula a distância dessa amostra para todas as outras amostras do banco de dados(a métrica utilizada para calcular a distância pode variar com base nos cenários de projeto.).</li>
<li>Filtra as K amostras mais próximas da amostra não classificada ainda.</li>
<li>Verifica a classe mais frequentes dentre as K amostras filtradas no passo 3.</li>
<li>Classifica a amostra do passo 1 como pertencente a classe obtida no passo 4.</li>
</ol>

### Implementacão do classificador "K-Nearest Neighbors (KNN)"

In [208]:
norm_p = lambda x,y,p : abs((x-y))**p
euclidian = lambda list1,list2: np.sqrt(sum(map(norm_p,list1,list2,[2]*len(list1))))
manhatan = lambda list1,list2: sum(map(norm_p,list1,list2,[1]*len(list1)))

metrics = {
    "euclidian" : euclidian,
    "manhatan" : manhatan
}

class KNNClassifier:
    def __init__(self, metric="euclidian", n_neighbors=3):
        if metric not in metrics:
            message = "invalid metric. the acceptable values are :"
            for k in metrics.keys():
                message += " " + k
            raise Exception(message)
        self.n_neighbors = n_neighbors
        self.metric_name = metric
        self.metric_func = metrics[metric]
    def fit(self, x_train,y_train):
        if len(x_train) != len(y_train):
            raise Exception("the size of inputs must be equals")
        self.x_train = x_train
        self.y_train = y_train
    def predict(self,x_test):
        distances = []
        result = []
        for test in x_test:
            for index in range(len(self.x_train)):
                distance = self.metric_func(self.x_train[index],test)
                distances.append((self.y_train[index], distance))
            distances = sorted(distances, key = lambda tup : tup[1])
            classes = collections.Counter(map(lambda x : x[0], distances[:self.n_neighbors]))
            clas = classes.most_common(1)
            result.append(clas[0][0])
            distances.clear()
        return result

## Parte 1.1 : KNearest Neighbors
### Avaliação

#### Leitura da base "demartology"

In [209]:
data = pandas.read_csv("./data/dermatology.csv", header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3


Converter o dataframe pandas para um objeto do numpy.

In [210]:
data = data.to_numpy()

#### Pré-processamento

Como informado anteriormente, a coluna 33 do banco de dados possui valores inválidos que atrapalham a execução do algoritmo, logo, afim de evitar a remoção da amostras que possuem esse problema, foi adotado a heurística de substituir os valores inválidos,pontos de interrogação,pela mediana da coluna. A mediana foi escolhida por ser menos sensível a outliers se comparada com a média, por exemplo.

In [211]:
data[:,33] = transform_column(data[:,33])
data = data.astype("float64")

Agora iremos separar o conjunto de dados em uma matriz de features e um array com as classes.

In [212]:
features = data[:,:34]
classes = data[:,34]

#### Execução do algoritmo (usando a distância EUCLIDIANA):

<ul>
    <li> Utilizando K = 7 </li>
    <li> Distância Euclidiana </li>
    <li> 80% do banco de dados como dados de TREINAMENTO </li>
    <li> 20% do banco de dados como dados de TESTE </li>
</ul>

#### Acurácia média para 100 rodadas do algoritmo.

In [213]:
results = []
size_tests = 100
KNN = KNNClassifier('euclidian',7)
for index in range(size_tests):
    x_train,x_test,y_train,y_test = train_test_split(features, target, test_size=0.2)
    KNN.fit(x_train,y_train)
    predictions = KNN.predict(x_test)
    results.append((predictions,y_test))

accuracies = []
for r in results:
    accuracies.append(accuracy_score(r[0],r[1]))
print("Acurácia média : {:.2f}%".format(np.mean(accuracies)*100))

Acurácia média : 85.91%


#### Acurácias médias de acerto por classe.

In [214]:
classes_all = set(classes)
for c in classes_all:
    score = []
    for r in results:
        score.append(accuracy_score_per_class(predictions,y_test,c))
    print("Classe {:} : {:.2f}%".format(c,np.mean(score)*100))
    score.clear()

Classe 1.0 : 96.00%
Classe 2.0 : 33.33%
Classe 3.0 : 100.00%
Classe 4.0 : 85.71%
Classe 5.0 : 100.00%
Classe 6.0 : 100.00%


#### Matrizes de confusão para o pior e melhor caso dentre as 100 rodadas de treinamente/teste.

In [215]:
acc = accuracy_score(results[0][0],results[0][1])
higher_value = acc
lower_value = acc
lower = results[0]
higher = results[0]
for index in range(1,len(results)):
    accuracy = accuracy_score(results[index][0],results[index][1])
    if accuracy > higher_value:
        higher = results[index]
        higher_value = accuracy
    if accuracy < lower_value:
        lower = results[index]
        lower_value = accuracy
        
print("Melhor caso - Acurácia: {:.2f}%".format(accuracy_score(higher[0],higher[1])*100))
print(confusion_matrix(higher[0],higher[1]))
print()
print("Pior caso - Acurácia : {:.2f}%".format(accuracy_score(lower[0],lower[1])*100))
print(confusion_matrix(lower[0],lower[1]))

Melhor caso - Acurácia: 97.30%
[[30.  0.  0.  0.  0.  0.]
 [ 0.  6.  0.  1.  0.  0.]
 [ 0.  0. 15.  0.  0.  0.]
 [ 0.  0.  0.  7.  0.  0.]
 [ 0.  0.  0.  0. 13.  0.]
 [ 1.  0.  0.  0.  0.  1.]]

Pior caso - Acurácia : 75.68%
[[19.  0.  0.  1.  4.  0.]
 [ 1.  5.  0.  4.  1.  0.]
 [ 0.  0. 20.  0.  0.  0.]
 [ 0.  3.  1.  4.  2.  0.]
 [ 0.  0.  0.  0.  5.  0.]
 [ 0.  0.  0.  0.  1.  3.]]


#### Execução do algoritmo (usando a distância MANHATAN): :

<ul>
    <li> Utilizando K = 7 </li>
    <li> Distância Manhatan </li>
    <li> 80% do banco de dados como dados de TREINAMENTO </li>
    <li> 20% do banco de dados como dados de TESTE </li>
</ul>

#### Acurácia média para 100 rodadas do algoritmo.

In [216]:
results = []
size_tests = 100
KNN = KNNClassifier('manhatan',7)
for index in range(size_tests):
    x_train,x_test,y_train,y_test = train_test_split(features, target, test_size=0.2)
    KNN.fit(x_train,y_train)
    predictions = KNN.predict(x_test)
    results.append((predictions,y_test))

accuracies = []
for r in results:
    accuracies.append(accuracy_score(r[0],r[1]))
print("Acurácia média : {:.2f}%".format(np.mean(accuracies)*100))

Acurácia média : 94.77%


#### Acurácias médias de acerto por classe.

In [217]:
classes_all = set(classes)
for c in classes_all:
    score = []
    for r in results:
        score.append(accuracy_score_per_class(predictions,y_test,c))
    print("Classe {:} : {:.2f}%".format(c,np.mean(score)*100))
    score.clear()

Classe 1.0 : 100.00%
Classe 2.0 : 85.71%
Classe 3.0 : 100.00%
Classe 4.0 : 100.00%
Classe 5.0 : 100.00%
Classe 6.0 : 100.00%


#### Matrizes de confusão para o pior e melhor caso dentre as 100 rodadas de treinamente/teste.

In [218]:
acc = accuracy_score(results[0][0],results[0][1])
higher_value = acc
lower_value = acc
lower = results[0]
higher = results[0]
for index in range(1,len(results)):
    accuracy = accuracy_score(results[index][0],results[index][1])
    if accuracy > higher_value:
        higher = results[index]
        higher_value = accuracy
    if accuracy < lower_value:
        lower = results[index]
        lower_value = accuracy
        
print("Melhor caso - Acurácia: {:.2f}%".format(accuracy_score(higher[0],higher[1])*100))
print(confusion_matrix(higher[0],higher[1]))
print()
print("Pior caso - Acurácia : {:.2f}%".format(accuracy_score(lower[0],lower[1])*100))
print(confusion_matrix(lower[0],lower[1]))

Melhor caso - Acurácia: 100.00%
[[28.  0.  0.  0.  0.  0.]
 [ 0. 10.  0.  0.  0.  0.]
 [ 0.  0. 15.  0.  0.  0.]
 [ 0.  0.  0.  7.  0.  0.]
 [ 0.  0.  0.  0. 10.  0.]
 [ 0.  0.  0.  0.  0.  4.]]

Pior caso - Acurácia : 89.19%
[[25.  0.  0.  0.  0.  0.]
 [ 0.  8.  0.  1.  0.  0.]
 [ 0.  0. 13.  0.  0.  0.]
 [ 0.  7.  0.  7.  0.  0.]
 [ 0.  0.  0.  0. 11.  0.]
 [ 0.  0.  0.  0.  0.  2.]]


### Conclusão

#### Podemos ver que a distância MANHATAN funciona melhor para esse conjunto de dados do que a distância EUCLIDIANA, pois obtemos uma acurácia média de 100%.

## Parte 2 : Linear Discriminant Analysis
### Funcionamento do algoritmo

O algoritmo conhecido como LDA (Linear Discriminant Analysis) é um algoritmo que pode ser utilizado tanto para problemas de classificação quanto para reduzir dimensionalidade em problemas com um extenso conjunto de dados. Quando utilizado como classificador, à exemplo da aplicação neste relatório, o objetivo principal do algoritmo é obter N funções discriminantes, sendo N igual ao número de classes no conjunto de dados, onde cada função está associada a uma classe do conjunto de dados. O discriminante com maior valor define a qual classe pertence a amostra analisada. Os passos para execução do algoritmo se encontram logo abaixo:

<ol>
<li>Recebe uma amostra que ainda não foi classificada.</li>
<li>Calcula o vetor médio da matriz de features.</li>
<li>Calcula as matrizes de covariância para cada classe usando suas respectivas amostras.</li>
<li>Calcula a matriz de espalhamento utilizando as matrizes de covariância.</li>
<li>Salva os termos obtidos anteriormente para cada discriminante associado a suas respectivas classes.</li>
<li>Para classificar a nova amostra, basta calcular os discriminantes para cada classe usando os termos obtidos no passo 4.O maior discriminantes,que está diretamente associado a uma classe, define a qual classe pertence a nova amostra.</li>
</ol>

### Implementacão do classificador "Linear Discriminant Analysis (LDA)"

In [219]:
class LDA:
    def __init__(self,features,target):
        self.features = features
        self.target = target
    def fit(self):
        global_mean = feature_mean(self.features)
        class_values = np.sort(np.unique(self.target))
        self.class_values = class_values
        cov_matrixes = []
        for c in class_values:
            cov = cov_matrix(self.features,self.target,c,global_mean)
            cov_matrixes.append(cov)
        self.mean_per_class = []
        for c in class_values:
            indexes = self.target == c
            mean_v = feature_mean(self.features[indexes])
            self.mean_per_class.append(mean_v)
        self.within = within_classes(self.features,cov_matrixes,self.target)
    def predict(self,new_data):
        values = []
        for idx,c in enumerate(self.class_values):
            disc = discrimant(self.mean_per_class[idx],self.within,new_data,self.target,c,len(self.features))
            values.append(disc)
        return self.class_values[values.index(max(values))]

## Parte 2.1 : Linear Discriminant Analysis(LDA)
### Avaliação

#### Leitura da base "demartology"

In [220]:
data = pandas.read_csv("./data/dermatology.csv", header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3


Converter o dataframe pandas para um objeto do numpy.

In [221]:
data = data.to_numpy()

#### Pré-processamento

Como informado anteriormente, a coluna 33 do banco de dados possui valores inválidos que atrapalham a execução do algoritmo, logo, afim de evitar a remoção da amostras que possuem esse problema, foi adotado a heurística de substituir os valores inválidos,pontos de interrogação,pela mediana da coluna. A mediana foi escolhida por ser menos sensível a outliers se comparada com a média, por exemplo.

In [222]:
data[:,33] = transform_column(data[:,33])
data = data.astype("float64")

Agora iremos separar o conjunto de dados em uma matriz de features e um array com as classes.

In [223]:
features = data[:,:34]
target = data[:,34]

#### Execução do algoritmo :

<ul>
    <li> 80% do banco de dados como dados de TREINAMENTO. </li>
    <li> 20% do banco de dados como dados de TESTE. </li>
    <li> Executar o algoritmo 100 vezes e obter a acurácia média. </li>
</ul>

In [224]:
ac = []
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2,random_state=42)
    lda = LDA(X_train,y_train)
    lda.fit()
    predicts = []
    for i in X_test:
        predicts.append(lda.predict(i))
    count = 0
    for i in range(len(y_test)):
        if y_test[i] == predicts[i]:
            count+=1
    ac.append(100*count/len(y_test))

print("Acurácia média : {:.2f}%".format(np.mean(ac))) 

Acurácia média : 93.24%
